In [284]:
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm,tnrange
class InvertedIndex():
    def __init__(self):
        self.DFpostings={}
        self.tokens_files=[]

    def __str__(self):
        return "Keys:"+str(len(list(self.db.keys())))

    def stripSpecialChar(self,text):
        return ''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation)

    def preProcess(self,text):
        stemmer = SnowballStemmer("english")
        stopWords = set(stopwords.words('english'))

        text = text.lower()                                     # convert all text to lower case
        text_tokens = word_tokenize(text)                       # tokenizing the text

        # stemmedWords = list([stemmer.stem(word) for word in text_tokens])
        # validTokens = [i for i in stemmedWords if i not in stopWords]

        validTokens = [i for i in text_tokens if i not in stopWords]    # removing stop words

        validTokens = [self.stripSpecialChar(x) for x in validTokens]   # stripping special characters
        validTokens = [x for x in validTokens if len(x) > 1]    # Choosing only words which has length > 1
        return validTokens, set(validTokens)

    def indexFile(self,file,fileId):
        '''
        Indexes the file pass as argument to the associated fileID
        '''
        tokens, setTokens = self.preProcess(file)
        self.tokens_files.append(tokens)
        for i in setTokens:
            if i in self.DFpostings:
                self.DFpostings[i].append(fileId)
            else:
                self.DFpostings[i] = [fileId]

    def generateWordcloud(self):
        '''
        Creates a wordclound to visualize the frequence of words in the index
        '''
        frequencyDict = {}
        for key in self.db:
            frequencyDict[key] = len(self.db[key])
        wordcloud = WordCloud().generate_from_frequencies(frequencyDict)
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.axis("off")
    
    def get_fileTokens(i):
        return self.tokens_files(i)

    def save(self,file):
        '''
        Save the index to a file locally
        '''
        json.dump(self.DFpostings, open(file, "w"))


In [285]:
import pickle
with open('PostingsAndTokens2.obj','rb') as file_object:
    raw_data = file_object.read()
work_obj = pickle.loads(raw_data)

In [286]:
class tf_idfmatrices():
    def __init__(self,DFpostings, tokens, docs_count):
        self.DFpostings = DFpostings
        self.tokens = tokens
        self.docs_count = docs_count
        self.vocab_count = len(DFpostings)
        self.vocabulary = [x for x in DFpostings]
        self.idf = dict()
        self.counter_lists = []
        self.most_common2 = []

        self.tf_idf_Binary = np.zeros((docs_count, self.vocab_count))
        self.tf_idf_RawCount = np.zeros((docs_count, self.vocab_count))
        self.tf_idf_TermFreq = np.zeros((docs_count, self.vocab_count))
        self.tf_idf_LogNorm = np.zeros((docs_count, self.vocab_count))
        self.tf_idf_DoubleNorm = np.zeros((docs_count, self.vocab_count))
    
    def generateIDF(self):
         for key in self.DFpostings:
                c = len(self.DFpostings[key])
                c=c+1                      #smoothihg factor
                self.idf[key] = np.log10(self.docs_count/c)
        
    def generateCounterLists(self):
        for i in tnrange(self.docs_count):
            self.counter_lists.append(Counter(self.tokens[i]))
            
    def generateMostCommon2(self):
        for i in tnrange(self.docs_count):
            self.most_common2.append((self.counter_lists[i]).most_common(2))
    
    def generateBinary(self):
        for i in tnrange(len(self.vocabulary)):
            for j in range(self.docs_count):
                if(self.vocabulary[i] in self.tokens[j]):
                    self.tf_idf_Binary[j][i]=1*self.idf[self.vocabulary[i]]
                    
    def generateRawCount(self):
        for i in tnrange(len(self.vocabulary)):
            for j in range(self.docs_count):
                self.tf_idf_RawCount[j][i] = self.counter_lists[j][self.vocabulary[i]]*self.idf[self.vocabulary[i]]
    
    def generateTermFreq(self):
        for i in tnrange(len(self.vocabulary)):
            for j in range(self.docs_count):
                self.tf_idf_TermFreq[j][i] = (self.counter_lists[j][self.vocabulary[i]]/len(self.tokens[j]))*self.idf[self.vocabulary[i]]
    
    def generateLogNorm(self):
        for i in tnrange(len(self.vocabulary)):
            for j in range(self.docs_count):
                self.tf_idf_LogNorm[j][i] = np.log10(1+self.counter_lists[j][self.vocabulary[i]])*self.idf[self.vocabulary[i]]
    
    def generateDoubleNorm(self):
        for i in tnrange(len(self.vocabulary)):
            for j in range(self.docs_count):
                maxnum=0;
                if(self.most_common2[j][0][0]==self.vocabulary[i]):
                    maxnum = self.most_common2[j][1][1]
                else:
                    maxnum = self.most_common2[j][0][1]            
                self.tf_idf_DoubleNorm[j][i] = (0.5+0.5*(self.counter_lists[j][self.vocabulary[i]]/maxnum))*self.idf[self.vocabulary[i]]

In [132]:
work_objthis = tf_idfmatrices(work_obj.DFpostings, work_obj.tokens_files,467)
work_objthis.generateIDF()
work_objthis.generateCounterLists()
work_objthis.generateMostCommon2()
work_objthis.generateBinary()
work_objthis.generateRawCount()
work_objthis.generateTermFreq()
work_objthis.generateLogNorm()
work_objthis.generateDoubleNorm()

  0%|          | 0/467 [00:00<?, ?it/s]

  0%|          | 0/467 [00:00<?, ?it/s]

  0%|          | 0/54867 [00:00<?, ?it/s]

  0%|          | 0/54867 [00:00<?, ?it/s]

  0%|          | 0/54867 [00:00<?, ?it/s]

  0%|          | 0/54867 [00:00<?, ?it/s]

  0%|          | 0/54867 [00:00<?, ?it/s]

In [287]:
# import pickle 
# serialized = pickle.dumps(work_objthis)
# filename = 'TF_IDF_Calculated.obj'
# with open(filename,'wb') as file_object:
#     file_object.write(serialized)


with open('TF_IDF_Calculated.obj','rb') as file_object:
    raw_data = file_object.read()
work_objthis = pickle.loads(raw_data)

In [288]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer,WordNetLemmatizer
import json
import os
from tqdm.notebook import tqdm,tnrange
import string
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import defaultdict
from tabulate import tabulate

nltk.download("punkt")
nltk.download("stopwords")
def stripSpecialChar(text):
        return ''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation)
    
def preProcess(text):
        stemmer = SnowballStemmer("english")
        stopWords = set(stopwords.words('english'))

        text = text.lower()                                     # convert all text to lower case
        text_tokens = word_tokenize(text)                       # tokenizing the text

        # stemmedWords = list([stemmer.stem(word) for word in text_tokens])
        # validTokens = [i for i in stemmedWords if i not in stopWords]

        validTokens = [i for i in text_tokens if i not in stopWords]    # removing stop words

        validTokens = [stripSpecialChar(x) for x in validTokens]   # stripping special characters
        validTokens = [x for x in validTokens if len(x) > 1]    # Choosing only words which has length > 1
        return validTokens

[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [289]:
class queryProcessing():
    def __init__(self, queryList, work_objthis):
        self.queryList = queryList
        self.work_objthis = work_objthis
        self.query_eval = np.zeros((work_objthis.docs_count,5))
    def compute_Scores(self):
        indextolookfor = []
        for word in self.queryList:
            index = self.work_objthis.vocabulary.index(word)
            indextolookfor.append(index)
        for docs in range(self.work_objthis.docs_count):
            for query in indextolookfor:
                self.query_eval[docs][0]+= self.work_objthis.tf_idf_Binary[docs][query]
                self.query_eval[docs][1]+= self.work_objthis.tf_idf_RawCount[docs][query]
                self.query_eval[docs][2]+= self.work_objthis.tf_idf_TermFreq[docs][query]
                self.query_eval[docs][3]+= self.work_objthis.tf_idf_LogNorm[docs][query]
                self.query_eval[docs][4]+= self.work_objthis.tf_idf_DoubleNorm[docs][query]
                
    def Top5(self,alist):
        return sorted(range(len(alist)), key=lambda i: alist[i], reverse=True)[:5]

    
    def results(self):
        print("For Binary TF-IDF, Top 5 documents:")
        print(self.Top5(self.query_eval[:,0]))
        print("")
        print("For RawCount TF-IDF, Top 5 documents:")
        print(self.Top5(self.query_eval[:,1]))
        print("")
        print("For TermFreq TF-IDF, Top 5 documents:")
        print(self.Top5(self.query_eval[:,2]))
        print("")
        print("For LogNorm TF-IDF, Top 5 documents:")
        print(self.Top5(self.query_eval[:,3]))
        print("")
        print("For DoubleNorm TF-IDF, Top 5 documents:")
        print(self.Top5(self.query_eval[:,4]))
        print("")

# TF-IDF Score

In [290]:
sentence_query = input("Enter the sentence:-")
listofwords = preProcess(sentence_query)
evaluation = queryProcessing(listofwords,work_objthis)
evaluation.compute_Scores()
evaluation.results()

Enter the sentence:-king kong
For Binary TF-IDF, Top 5 documents:
[276, 402, 6, 8, 15]

For RawCount TF-IDF, Top 5 documents:
[214, 230, 26, 160, 169]

For TermFreq TF-IDF, Top 5 documents:
[286, 335, 271, 230, 160]

For LogNorm TF-IDF, Top 5 documents:
[214, 230, 26, 160, 169]

For DoubleNorm TF-IDF, Top 5 documents:
[160, 230, 286, 15, 399]



# Cosine - Similarity

In [291]:
query_obj = tf_idfmatrices(work_obj.DFpostings, [listofwords], 1)
query_obj.idf = work_objthis.idf

### Do Not Uncomment!!! ####
# query_obj.generateIDF()

query_obj.generateCounterLists()
query_obj.generateMostCommon2()
query_obj.generateBinary()
query_obj.generateRawCount()
query_obj.generateTermFreq()
query_obj.generateLogNorm()
query_obj.generateDoubleNorm()

In [292]:
# print(query_obj.tf_idf_Binary.shape)
# print(query_obj.tf_idf_RawCount.shape)
# print(query_obj.tf_idf_TermFreq.shape)
# print(query_obj.tf_idf_LogNorm.shape)
# print(query_obj.tf_idf_DoubleNorm.shape)

In [293]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [294]:
def Cosine_Similarity(Q,D,scheme):
    """
    Q -> query vector (1 x vocab_count)
    D -> document vector (n_doc x vocab_count)
    """
    n_doc = D.shape[0]
    Z = np.zeros((n_doc, 1))
    
    Q_ = Q.T
    for i in range(n_doc):
        Z[i,0] = dot(D[i,:],Q_[:,0]) / ( norm(D[i,:]) * norm(Q_[:,0]))
    
    doc_str = [str(i) for i in range(n_doc)]
    
    sim = list(zip(Z[:,0],doc_str))
    sim.sort(reverse=True)
    files = [int(x[1]) for x in sim[:5]]
    scores = [float(x[0]) for x in sim[:5]]
    
    print("------ Scheme :: " + scheme + " ------")
    print("------ Top 5 Matching Documents ------")    
    
    mapping = json.load(open('mapping.json'))
    
    data = [[i,mapping[i]] for i in files]
    for i in range(len(data)):
        data[i].append(scores[i])

    print(tabulate(data,headers=['Document ID','Location','Scores']))

### Cosine Similarity - Binary scheme

In [295]:
Cosine_Similarity(query_obj.tf_idf_Binary,work_objthis.tf_idf_Binary,scheme="Binary")

------ Scheme :: Binary ------
------ Top 5 Matching Documents ------
  Document ID  Location                 Scores
-------------  --------------------  ---------
          276  stories/mario.txt     0.0833187
          402  stories/superg1       0.0330532
          267  stories/lionwar.txt   0.0234361
          266  stories/lionmosq.txt  0.0220713
          186  stories/foxncrow.txt  0.0218335


### Cosine Similarity - Raw Count

In [296]:
Cosine_Similarity(query_obj.tf_idf_RawCount,work_objthis.tf_idf_RawCount,scheme="Raw Count")

------ Scheme :: Raw Count ------
------ Top 5 Matching Documents ------
  Document ID  Location                 Scores
-------------  --------------------  ---------
          160  stories/fable.txt     0.117795
          230  stories/hop-frog.poe  0.0962945
          286  stories/monkking.txt  0.090235
           15  stories/6ablemen.txt  0.0801088
          335  stories/pussboot.txt  0.0708097


### Cosine Similarity - Term Frequency

In [297]:
Cosine_Similarity(query_obj.tf_idf_TermFreq,work_objthis.tf_idf_TermFreq,scheme="Term Frequency")

------ Scheme :: Term Frequency ------
------ Top 5 Matching Documents ------
  Document ID  Location                 Scores
-------------  --------------------  ---------
          160  stories/fable.txt     0.117795
          230  stories/hop-frog.poe  0.0962945
          286  stories/monkking.txt  0.090235
           15  stories/6ablemen.txt  0.0801088
          335  stories/pussboot.txt  0.0708097


### Cosine Similarity - Log Normalization

In [298]:
Cosine_Similarity(query_obj.tf_idf_LogNorm,work_objthis.tf_idf_LogNorm,scheme="Log Normalization")

------ Scheme :: Log Normalization ------
------ Top 5 Matching Documents ------
  Document ID  Location                 Scores
-------------  --------------------  ---------
          276  stories/mario.txt     0.0990284
          286  stories/monkking.txt  0.0549539
          335  stories/pussboot.txt  0.041418
           15  stories/6ablemen.txt  0.0399768
          271  stories/lpeargrl.txt  0.0395454


### Cosine Similarity - Double Normalization

In [299]:
Cosine_Similarity(query_obj.tf_idf_DoubleNorm,work_objthis.tf_idf_DoubleNorm,scheme="Double Normalization")

------ Scheme :: Double Normalization ------
------ Top 5 Matching Documents ------
  Document ID  Location               Scores
-------------  -------------------  --------
          167  stories/fear.hum     0.999988
          287  stories/monksol.txt  0.999988
           53  stories/bestwish     0.999987
           96  stories/bumm         0.999987
          293  stories/myeyes       0.999987
